   * https://blog.min.io/kafka_and_minio/: Publish from Kafka, Persist on MinIO
   * https://docs.confluent.io/kafka-connectors/s3-sink/current/overview.html#
      * Search "Quick Start"  

In [2]:
!kubectl -n kafka-main-cluster get kafkaconnector s3-sink-cdc -o yaml | yq

apiVersion: kafka.strimzi.io/v1beta2
kind: KafkaConnector
metadata:
  annotations:
    kubectl.kubernetes.io/last-applied-configuration: |
      {"apiVersion":"kafka.strimzi.io/v1beta2","kind":"KafkaConnector","metadata":{"annotations":{},"labels":{"strimzi.io/cluster":"my-connect"},"name":"s3-sink-cdc","namespace":"kafka-main-cluster"},"spec":{"class":"io.confluent.connect.s3.S3SinkConnector","config":{"flush.size":3,"format.class":"io.confluent.connect.s3.format.avro.AvroFormat","key.converter":"io.confluent.connect.avro.AvroConverter","key.converter.schema.registry.url":"http://main-registry-schema-registry.kafka-main-cluster.svc:8081","partitioner.class":"io.confluent.connect.storage.partitioner.DefaultPartitioner","s3.bucket.name":"bronze","schema.compatibility":"NONE","schema.generator.class":"io.confluent.connect.storage.hive.schema.DefaultSchemaGenerator","schema.registry.url":"http://main-registry-schema-registry.kafka-main-cluster.svc:8081","storage.class":"io.confluent.conne

In [21]:
!kubectl -n kafka-main-cluster get kafkaconnector s3-sink-cdc -o jsonpath='{.status.conditions[0].message}'

The following tasks have failed: 0, see connectorStatus for more details.

In [3]:
!kubectl -n kafka-main-cluster get kafkaconnector s3-sink-cdc -o jsonpath='{.status.connectorStatus.tasks[0].trace}'

org.apache.kafka.connect.errors.ConnectException: com.amazonaws.SdkClientException: Unable to execute HTTP request: PKIX path building failed: sun.security.provider.certpath.SunCertPathBuilderException: unable to find valid certification path to requested target
	at io.confluent.connect.s3.S3SinkTask.start(S3SinkTask.java:142)
	at org.apache.kafka.connect.runtime.WorkerSinkTask.initializeAndStart(WorkerSinkTask.java:329)
	at org.apache.kafka.connect.runtime.WorkerTask.doRun(WorkerTask.java:202)
	at org.apache.kafka.connect.runtime.WorkerTask.run(WorkerTask.java:259)
	at org.apache.kafka.connect.runtime.isolation.Plugins.lambda$withClassLoader$1(Plugins.java:236)
	at java.base/java.util.concurrent.Executors$RunnableAdapter.call(Executors.java:539)
	at java.base/java.util.concurrent.FutureTask.run(FutureTask.java:264)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1136)
	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExe

In [15]:
!echo | openssl s_client -connect minio.minio-tenant-1.svc:443

Connecting to 10.105.224.249
CONNECTED(00000003)
depth=0 O=system:nodes, CN=system:node:*.minio-tenant-1-hl.minio-tenant-1.svc.cluster.local
verify error:num=20:unable to get local issuer certificate
verify return:1
depth=0 O=system:nodes, CN=system:node:*.minio-tenant-1-hl.minio-tenant-1.svc.cluster.local
verify error:num=21:unable to verify the first certificate
verify return:1
depth=0 O=system:nodes, CN=system:node:*.minio-tenant-1-hl.minio-tenant-1.svc.cluster.local
verify return:1
---
Certificate chain
 0 s:O=system:nodes, CN=system:node:*.minio-tenant-1-hl.minio-tenant-1.svc.cluster.local
   i:CN=minikubeCA
   a:PKEY: id-ecPublicKey, 256 (bit); sigalg: RSA-SHA256
   v:NotBefore: Feb  7 10:50:52 2024 GMT; NotAfter: Feb  6 10:50:52 2025 GMT
---
Server certificate
-----BEGIN CERTIFICATE-----
MIIDnjCCAoagAwIBAgIPU5kD6xSa/ngVOQZSvgM7MA0GCSqGSIb3DQEBCwUAMBUx
EzARBgNVBAMTCm1pbmlrdWJlQ0EwHhcNMjQwMjA3MTA1MDUyWhcNMjUwMjA2MTA1
MDUyWjBiMRUwEwYDVQQKEwxzeXN0ZW06bm9kZXMxSTBHBgNVBAMMQHN5c3RlbTpu

In [14]:
!echo | openssl s_client  -connect minio.minio-tenant-1.svc:443 2> /dev/null \
      | sed -ne '/-BEGIN CERTIFICATE-/,/-END CERTIFICATE-/p'

-----BEGIN CERTIFICATE-----
MIIDnjCCAoagAwIBAgIPU5kD6xSa/ngVOQZSvgM7MA0GCSqGSIb3DQEBCwUAMBUx
EzARBgNVBAMTCm1pbmlrdWJlQ0EwHhcNMjQwMjA3MTA1MDUyWhcNMjUwMjA2MTA1
MDUyWjBiMRUwEwYDVQQKEwxzeXN0ZW06bm9kZXMxSTBHBgNVBAMMQHN5c3RlbTpu
b2RlOioubWluaW8tdGVuYW50LTEtaGwubWluaW8tdGVuYW50LTEuc3ZjLmNsdXN0
ZXIubG9jYWwwWTATBgcqhkjOPQIBBggqhkjOPQMBBwNCAATRFxBTliVvPJKE6HM1
o53bCeCPFrQsXg7s9NN1fjW+24Cn2NifkMtA2yXWkK9eQL5ffwBBJw+E0Do8VIT6
/7kRo4IBZzCCAWMwDgYDVR0PAQH/BAQDAgWgMBMGA1UdJQQMMAoGCCsGAQUFBwMB
MAwGA1UdEwEB/wQCMAAwHwYDVR0jBBgwFoAU3Ov1YUl7wV+DyUwXCZTM+pWtalkw
ggELBgNVHREEggECMIH/gk5taW5pby10ZW5hbnQtMS1zcy0wLXswLi4uMn0ubWlu
aW8tdGVuYW50LTEtaGwubWluaW8tdGVuYW50LTEuc3ZjLmNsdXN0ZXIubG9jYWyC
JW1pbmlvLm1pbmlvLXRlbmFudC0xLnN2Yy5jbHVzdGVyMi54cHSCFG1pbmlvLm1p
bmlvLXRlbmFudC0xghhtaW5pby5taW5pby10ZW5hbnQtMS5zdmOCMyoubWluaW8t
dGVuYW50LTEtaGwubWluaW8tdGVuYW50LTEuc3ZjLmNsdXN0ZXIyLnhwdIIhKi5t
aW5pby10ZW5hbnQtMS5zdmMuY2x1c3RlcjIueHB0MA0GCSqGSIb3DQEBCwUAA4IB
AQCfgkhWGmxr0/JkPC7hjg8WDyR2ncjRk1B87UFsLwsHWf/isC90vARbj8CUtd